<a href="https://colab.research.google.com/github/hyungillee/NaverLabsChallenge/blob/main/%EB%84%B7%EB%B8%8C1f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Aug 23 10:05:50 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
import torch
import torchvision.transforms as transforms
import torch.utils.data as data
import os
from os.path import join, exists
from scipy.io import loadmat
import numpy as np
from random import randint, random
from collections import namedtuple
from PIL import Image

from sklearn.neighbors import NearestNeighbors
import h5py
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np
import time


In [ ]:
!apt install libomp-dev #코랩에서 faiss 설치
!python -m pip install --upgrade faiss faiss-gpu
import faiss

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Requirement already up-to-date: faiss in /usr/local/lib/python3.6/dist-packages (1.5.3)
Requirement already up-to-date: faiss-gpu in /usr/local/lib/python3.6/dist-packages (1.6.3)


In [ ]:

from __future__ import print_function
import argparse
from math import log10, ceil
import random, shutil, json
from os.path import join, exists, isfile, realpath, dirname
from os import makedirs, remove, chdir, environ

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader#, SubsetRandomSampler
#from torch.utils.data.dataset import Subset
import torchvision.transforms as transforms
from PIL import Image
from datetime import datetime
import torchvision.datasets as datasets
import torchvision.models as models
import h5py
#import faiss

#from tensorboardX import SummaryWriter
import numpy as np
#import netvlad

In [ ]:
class NetVLAD(nn.Module):
    """NetVLAD layer implementation"""

    def __init__(self, num_clusters=64, dim=128, 
                 normalize_input=True, vladv2=False):
        """
        Args:
            num_clusters : int
                The number of clusters
            dim : int
                Dimension of descriptors
            alpha : float
                Parameter of initialization. Larger value is harder assignment.
            normalize_input : bool
                If true, descriptor-wise L2 normalization is applied to input.
            vladv2 : bool
                If true, use vladv2 otherwise use vladv1
        """
        super(NetVLAD, self).__init__()
        self.num_clusters = num_clusters
        self.dim = dim # 512
        self.alpha = 0
        self.vladv2 = vladv2
        self.normalize_input = normalize_input
        self.conv = nn.Conv2d(dim, num_clusters, kernel_size=(1, 1), bias=vladv2)
        self.centroids = nn.Parameter(torch.rand(num_clusters, dim)) # [64,512] 행렬 
    def init_params(self, clsts, traindescs): #pretrained 시 필요없음
        print("init_params 실행됨")
        #TODO replace numpy ops with pytorch ops
        if self.vladv2 == False:
            clstsAssign = clsts / np.linalg.norm(clsts, axis=1, keepdims=True)
            dots = np.dot(clstsAssign, traindescs.T)
            dots.sort(0)
            dots = dots[::-1, :] # sort, descending

            self.alpha = (-np.log(0.01) / np.mean(dots[0,:] - dots[1,:])).item()
            self.centroids = nn.Parameter(torch.from_numpy(clsts))
            self.conv.weight = nn.Parameter(torch.from_numpy(self.alpha*clstsAssign).unsqueeze(2).unsqueeze(3))
            self.conv.bias = None
        else:
            knn = NearestNeighbors(n_jobs=-1) #TODO faiss?
            knn.fit(traindescs)
            del traindescs
            dsSq = np.square(knn.kneighbors(clsts, 2)[1])
            del knn
            self.alpha = (-np.log(0.01) / np.mean(dsSq[:,1] - dsSq[:,0])).item()
            self.centroids = nn.Parameter(torch.from_numpy(clsts))
            del clsts, dsSq

            self.conv.weight = nn.Parameter(
                (2.0 * self.alpha * self.centroids).unsqueeze(-1).unsqueeze(-1)
            )
            self.conv.bias = nn.Parameter(
                - self.alpha * self.centroids.norm(dim=1)
            )

    def forward(self, x):
        # print("변수확인 clusters:{0} dim:{1} alpha:{2} vladv2:{3} nomalize:{4} conv:{5} centroids:{6}".format(self.num_clusters,                                                                                               
        # self.dim, 
        # self.alpha,
        # self.vladv2,
        # self.normalize_input ,
        # self.conv,  
        # self.centroids ))

# 변수확인 clusters:64 dim:512 alpha:0 vladv2:True nomalize:True conv:Conv2d(512, 64, kernel_size=(1, 1), stride=(1, 1)) centroids:Parameter containing:
# tensor([[-0.0281,  0.0005, -0.0283,  ..., -0.0399, -0.0251, -0.0411],
#         [-0.0338, -0.0183, -0.0144,  ..., -0.0375, -0.0327, -0.0307],
#         [-0.0346, -0.0182, -0.0120,  ..., -0.0519, -0.0445, -0.0391],
#         ...,
#         [-0.0208, -0.0251, -0.0088,  ..., -0.0557, -0.0339, -0.0455],
#         [-0.0293, -0.0132, -0.0418,  ..., -0.0424, -0.0317, -0.0577],
#         [-0.0410, -0.0449, -0.0180,  ..., -0.0770, -0.0443, -0.0489]],
#        device='cuda:0', requires_grad=True)

        N, C = x.shape[:2] #4, 512
        print("@@@@@ netvlad @@@@@ ")
        print("x.shape {}, N :{} , C:{}".format(x.shape,N,C) ) # 4,512,8,8  N:4 C:512

        if self.normalize_input:
            x = F.normalize(x, p=2, dim=1)  # across descriptor dim , 입력값의 L2 norm 

        # soft-assignment
        soft_assign = self.conv(x).view(N, self.num_clusters, -1) #4,64,64
        print('soft_assign.shape :',soft_assign.shape) # 4,64,64
        soft_assign = F.softmax(soft_assign, dim=1) #cluster별로 softmax 

        x_flatten = x.view(N, C, -1) #4,512,64 행렬의 input값
        print('x_flatten shape:',x_flatten.shape) # 4,512,64
        # calculate residuals to each clusters
        vlad = torch.zeros([N, self.num_clusters, C], dtype=x.dtype, layout=x.layout, device=x.device) #4 , 64 , 512 
        print('dtype , layout : ',x.dtype, x.layout) # float32, torch.strided

        print("x_flatten.unsqueeze.permute",x_flatten.unsqueeze(0).permute(1, 0, 2, 3).shape ) #x_flatten.unsqueeze.permute torch.Size([4, 1, 512, 64])

        for C in range(self.num_clusters): # slower than non-looped, but lower memory usage #64 
            residual = x_flatten.unsqueeze(0).permute(1, 0, 2, 3) - \ #([4, 1, 512, 64])
                    self.centroids[C:C+1, :].expand(x_flatten.size(-1), -1, -1).permute(1, 2, 0).unsqueeze(0) #([1, 1, 512, 64])
            if C <10: # [64,512] -> 
                print('centroids shape: ',self.centroids[C:C+1, :].expand(x_flatten.size(-1), -1, -1).permute(1, 2, 0).unsqueeze(0).shape ) # centroids shape:  torch.Size([1, 1, 512, 64])

            residual *= soft_assign[:,C:C+1,:].unsqueeze(2)
            vlad[:,C:C+1,:] = residual.sum(dim=-1)
        
        print('vlad shape',vlad.shape) # 4,64,512
        

        
        vlad = F.normalize(vlad, p=2, dim=2)  # intra-normalization  , a few large components of the VLAD vector can adversely
        vlad = vlad.view(x.size(0), -1)  # flatten
        vlad = F.normalize(vlad, p=2, dim=1)  # L2 normalize
        #print("vlad 출력값:",vlad) vlad return에는 값들이 계속 나오는듯함
        return vlad

fiass로 유사한 이미지를 찾고  좌표값에 KNN을 돌린 결과를 이용해 정확도를 구한다

In [ ]:
def MakeFeature(test_data_loader,eval_set,epoch=0 ):         #write_tboard=False     #Netvlad에는 레이블이 안쓰인다
    # TODO what if features dont fit in memory? 
  start = time.time()

  model.eval()
  with torch.no_grad():
      print('====> Extracting Features')
      pool_size = 512 #encoder_dim ###
      pool_size *= 64 #opt.num_clusters   32768
      dbFeat = np.empty((len(eval_set), pool_size)) #(데이터크기, 32768) 의 행렬
      print("dbFeat의 shape: ",dbFeat.shape)
      a,b,c,d=0,1,2,3      
      for iteration, (input, indices) in enumerate(test_data_loader, 1): 
          
          #print("indices : ",indices , indices.detach().numpy()) 레이블값 [1,0,1,0] 등으로 나옴
          input = input.to(device)
          image_encoding = model.encoder(input)
          vlad_encoding = model.pool(image_encoding)#이미지 넣어서 netvlad까지 돌림 
          #print("vlad_encoding 값 : ",vlad_encoding.detach().cpu().numpy()) 
          #print("vlad의 shape",vlad_encoding.detach().cpu().numpy().shape) # vlad의 shape (4, 32768)

          batchsizelist=[a,b,c,d]
          if a>=len(eval_set)-1: #인덱스값 정확하게 27245와 나누어떨어져야함
            del batchsizelist[1:]
            print("마지막인덱스1개")
          elif b>=len(eval_set)-1:
            del batchsizelist[2:]
            print("마지막인덱스2개")
          elif c>=len(eval_set)-1:
            del batchsizelist[3]
            print("마지막인덱스3개")

          #print("indice numpy 값: ",batchsizelist) # indice numpy 값:  [0 0 0 0]
          
          dbFeat[batchsizelist, :] = vlad_encoding.detach().cpu().numpy() #각 데이터베이스에 netvlad까지 돌리는듯. cpu로
          
          #print("dbFeat의 인덱스를보자 : ",dbFeat[batchsizelist,0:5])
          if iteration % 50 == 0 or len(test_data_loader) <= 10:
              print("==> Batch ({}/{})".format(iteration, len(test_data_loader)), flush=True)
              print("dbFeat 걸린시간",time.time()-start)
          
          a+=4
          b+=4
          c+=4
          d+=4
          del input, image_encoding, vlad_encoding
  del test_data_loader
  return dbFeat    

def test(dbFeat,qFeat):#getPos numQ
    # qFeat = dbFeat[eval_set.dbStruct.numDb:].astype('float32')
    # dbFeat = dbFeat[:eval_set.dbStruct.numDb].astype('float32') #전부 학습후 db 이미지랑 쿼리 이미지 나눔
    pool_size=512*64
    print('====> Building faiss index')# db에서 쿼리이미지 유사한거 찾아주는 기능
    faiss_index = faiss.IndexFlatL2(pool_size)
    faiss_index.add(dbFeat)

    print('====> Calculating recall @ N')
    n_values = [1,5,10,20]
    print("max",n_values)
    _, predictions = faiss_index.search(qFeat, max(n_values)) 
    return predictions
##################################################################
    # # for each query get those within threshold distance
    # #gt = eval_set.getPositives()  #positive 함수사용함
    # gt = getPos

    # correct_at_n = np.zeros(len(n_values))
    # #TODO can we do this on the matrix in one go?
    # for qIx, pred in enumerate(predictions):
    #     for i,n in enumerate(n_values):
    #         # if in top N then also in top NN, where NN > N
    #         if np.any(np.in1d(pred[:n], gt[qIx])):     #np.any : 배열 데이터 조건 비교해서 조건이 맞으면 True     np.in1d: 첫번째 배열이 두번깨 배열의 원소를 포함하는지 여부
    #             correct_at_n[i:] += 1
    #             break
    # recall_at_n = correct_at_n / numQ

    # recalls = {} #make dict for output
    # for i,n in enumerate(n_values):
    #     recalls[n] = recall_at_n[i]
    #     print("====> Recall@{}: {:.4f}".format(n, recall_at_n[i]))
    #     if write_tboard: writer.add_scalar('Val/Recall@' + str(n), recall_at_n[i], epoch)

    # return recalls

In [ ]:
def test(dbFeat,qFeat):#getPos numQ
    # qFeat = dbFeat[eval_set.dbStruct.numDb:].astype('float32')
    # dbFeat = dbFeat[:eval_set.dbStruct.numDb].astype('float32') #전부 학습후 db 이미지랑 쿼리 이미지 나눔
    pool_size=512*64
    print('====> Building faiss index')# db에서 쿼리이미지 유사한거 찾아주는 기능
    faiss_index = faiss.IndexFlatL2(pool_size)
    print("faiss_index.is_trained : ",faiss_index.is_trained)
    faiss_index.add(dbFeat)
    print("faiss_index.ntotal : ",faiss_index.ntotal)
    print('====> Calculating recall @ N')
    n_values = [1,5,10,20]

    d, predictions = faiss_index.search(qFeat, n_values[2])   # d: ? 
    return predictions

**데이터 가져오기**

In [ ]:
num_workers=2
batchsize=4
def input_transform():
    return transforms.Compose([
        transforms.Resize((128,128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    ])


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
seed=123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    print("쿠다가능")
    torch.cuda.manual_seed(seed) 
else: print("쿠다없음")
#'/gdrive/My Drive/dataset/vardata/1fb1train'
# /gdrive/My Drive/1fdataset/1ftrainimages
train_imagenet_data =datasets.ImageFolder('/gdrive/My Drive/1fdataset/1ftrainimages', transform=input_transform())
train_data_loader = torch.utils.data.DataLoader(train_imagenet_data,
                                          batch_size=batchsize,
                                          shuffle=False,
                                          num_workers=num_workers)
#'/gdrive/My Drive/dataset/vardata/1fb1test'
#/gdrive/My Drive/1fdataset/1ftest
test_imagenet_data =datasets.ImageFolder('/gdrive/My Drive/1fdataset/1ftest/', transform=input_transform())
test_data_loader = torch.utils.data.DataLoader(test_imagenet_data,
                                          batch_size=batchsize,
                                          shuffle=False,
                                          num_workers=num_workers)
print(len(train_imagenet_data),len(train_data_loader))
print(len(test_imagenet_data),len(test_data_loader))

쿠다가능
27245 6812
1969 493


# 새 섹션

In [ ]:
# for idx, (data, label) in enumerate(train_data_loader):
#   if idx<100:
#     print("{} {} ",data.shape,label)

In [ ]:
# positives = None
# distance = None
# numdb=len(train_data_loader)
# numQ=len(test_data_loader)
# posDistThr=25 # 일단 이렇게 해보자
# posDistsqthr = 625
# nonTrivPos=100
# def getPositives(utmDb,utmQ,posDistThr): #좌표값 비교해서 가장 유사한 거리를 찾기 . posDistThr 
#     # positives for evaluation are those within trivial threshold range
#     #fit NN to find them, search by radius
#     if positives is None:
#         knn = NearestNeighbors(n_jobs=-1)
#         knn.fit(self.utmDb)
#         distances,positives = knn.radius_neighbors(utmQ,radius=posDistThr)

#     return self.positives

**CNN모델만들기**

In [ ]:
print('===> Building model')
encoder = models.vgg16(pretrained=True)
encoder_dim=512
layers = list(encoder.features.children())
print(layers[-1],layers[-2])
layers = list(encoder.features.children())[:-2]
print(layers[-1],layers[-2],layers[-3],layers[-4],layers[-5])
    # if using pretrained then only train conv5_1, conv5_2, and conv5_3
for l in layers[:-5]: 
    for p in l.parameters():
        p.requires_grad = False

encoder = nn.Sequential(*layers)
model = nn.Module() 
model.add_module('encoder', encoder)

===> Building model
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False) ReLU(inplace=True)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) ReLU(inplace=True) Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) ReLU(inplace=True) Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


**NETVLAD 풀링 합치기**

In [ ]:
num_clusters=64
net_vlad = NetVLAD(num_clusters=num_clusters, dim=encoder_dim, vladv2=True)
model.add_module('pool', net_vlad)

isParallel = False
nGPU=1
if nGPU > 1 and torch.cuda.device_count() > 1:
    print("실행")
    model.encoder = nn.DataParallel(model.encoder)
    model.pool = nn.DataParallel(model.pool)
    isParallel = True

print("load 전모델 상태")
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())

load 전모델 상태


**Checkpoint에는 pool의 bias값이 없네**

In [ ]:
# print("checkpoint의 파라미터 ")
# checkpoint = torch.load(ckpt, map_location=lambda storage, loc: storage)
# for param_tensor in checkpoint['state_dict']:
#     print(param_tensor)
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())

**Checkpoint 가져와서 모델에 넣기**

In [ ]:
ckpt = '/gdrive/My Drive//checkpoints/checkpoint.pth.tar'
checkpoint = torch.load(ckpt, map_location=lambda storage, loc: storage)
start_epoch=checkpoint['epoch']
best_metric = checkpoint['best_score']
print(start_epoch, best_metric)
print(checkpoint.keys())
model.load_state_dict(checkpoint['state_dict'],strict=False) #strict=False  일치하지 않는 키들을 무시하

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device',device)
model = model.to(device)

print(model)

print("=> loaded checkpoint '{}' (epoch {})".format(ckpt, checkpoint['epoch']))

25 0.9479495268138801
dict_keys(['epoch', 'state_dict', 'recalls', 'best_score', 'optimizer', 'parallel'])
device cuda:0
Module(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), str

In [ ]:
print('===> Running evaluation step')
epoch = 1
print(len(train_imagenet_data))
dbFeat=MakeFeature(train_data_loader, train_imagenet_data,epoch ) #Float 32 인지 확인
qFeat=MakeFeature(test_data_loader, test_imagenet_data,epoch)
#getPos=getPositives(utmDb,utmQ,posDistThr) #아직 정리해야됨
print(dbFeat.shape,qFeat.shape)
dbFeat=dbFeat.astype('float32') #float32 로 바꿔야 fiass가 돌아감
qFeat=qFeat.astype('float32')
recalls = test(dbFeat,qFeat)#getPos numQ
print(recalls , recalls.shape)  
print(recalls[0])
import csv
with open('/gdrive/My Drive/dataset/1fprediction.csv', 'w') as fileWrite:
  myWriter = csv.writer(fileWrite) 
  for i in recalls: 
      myWriter.writerow(i)

===> Running evaluation step
27245
====> Extracting Features
dbFeat의 shape:  (27245, 32768)
@@@@@ netvlad @@@@@ 
x.shape torch.Size([4, 512, 8, 8]), N :4 , C:512
soft_assign.shape : torch.Size([4, 64, 64])
x_flatten shape: torch.Size([4, 512, 64])
dtype , layout :  torch.float32 torch.strided
x_flatten.unsqueeze.permute torch.Size([4, 1, 512, 64])
centroids shape:  torch.Size([1, 1, 512, 64])
centroids shape:  torch.Size([1, 1, 512, 64])
centroids shape:  torch.Size([1, 1, 512, 64])
centroids shape:  torch.Size([1, 1, 512, 64])
centroids shape:  torch.Size([1, 1, 512, 64])
centroids shape:  torch.Size([1, 1, 512, 64])
centroids shape:  torch.Size([1, 1, 512, 64])
centroids shape:  torch.Size([1, 1, 512, 64])
centroids shape:  torch.Size([1, 1, 512, 64])
centroids shape:  torch.Size([1, 1, 512, 64])
vlad shape torch.Size([4, 64, 512])
@@@@@ netvlad @@@@@ 
x.shape torch.Size([4, 512, 8, 8]), N :4 , C:512
soft_assign.shape : torch.Size([4, 64, 64])
x_flatten shape: torch.Size([4, 512, 64]

KeyboardInterrupt: ignored

val data , worker=2  406, 95 초   or  662, 140 

In [ ]:
print(dbFeat.shape,qFeat.shape)
dbFeat=dbFeat.astype('float32') #float32 로 바꿔야 fiass가 돌아감
qFeat=qFeat.astype('float32')

NameError: ignored

In [ ]:
recalls = test(dbFeat,qFeat)#getPos numQ

NameError: ignored

In [ ]:
print(recalls , recalls.shape)  
print(recalls[0])

In [ ]:
import csv
with open('/gdrive/My Drive/dataset/1fprediction.csv', 'w') as fileWrite:
  myWriter = csv.writer(fileWrite) 
  for i in recalls: 
      myWriter.writerow(i)

Sun Jun 28 00:54:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**메모리 터지는거 같으니 **따로하자** **

In [ ]:
from PIL import Image
import torchvision.transforms.functional as Function
import torchvision.datasets as datasets
from IPython.display import display

In [ ]:
def input_transform2():
    return transforms.Compose([
        transforms.Resize((128,128)),  #전체데이터 한번에 가져올때 resize안해주면 메모리터짐
        transforms.ToTensor()        
    ])
bef_train_imagenet_data = datasets.ImageFolder('/gdrive/My Drive/1fdataset/1ftrainimages', transform=input_transform2())

bef_train_data_loader = torch.utils.data.DataLoader(bef_train_imagenet_data,
                                          batch_size=len(bef_train_imagenet_data),
                                          shuffle=False,
                                          num_workers=0)
bef_test_imagenet_data = datasets.ImageFolder('/gdrive/My Drive/1fdataset/1ftest/', transform=input_transform2())
bef_test_data_loader = torch.utils.data.DataLoader(bef_test_imagenet_data,
                                          batch_size=len(bef_test_imagenet_data),
                                          shuffle=False,
                                          num_workers=0)

In [ ]:
start = time.time()
for idx , (bef_train_image,_) in enumerate(bef_train_data_loader):
  bef_train_image2 = bef_train_image
print(":",time.time()-start)

KeyboardInterrupt: ignored

In [ ]:
check_index=0
dataiter = iter(bef_test_data_loader)
data,label=next(dataiter)
test_img = Function.to_pil_image(data[check_index])
display(test_img)

In [ ]:
for i in range(len(recalls)):
  print("i번째 : ",recalls[i])
  img = Function.to_pil_image(bef_train_image2[recalls[i]])
  display(img)

In [ ]:
import numpy as np
concat=np.empty((len(bef_test_imagenet_data),21))
concat=concat.tolist()
for i in range(len(bef_test_imagenet_data)):
  concat[i][0]=bef_test_imagenet_data.imgs[i][0].split('var1f/')[1]
  for j in range(20):
    concat[i][j+1]= bef_train_data_loader.imgs[recalls[i][j-1]][0].split('var1f/')[1]  

with open('/gdrive/My Drive/dataset/1fconcat.csv', 'w') as fileWrite:
  myWriter = csv.writer(fileWrite) 
  for i in concat: 
      myWriter.writerow(i)

In [ ]:
import csv
with open('/gdrive/My Drive/dataset/1fdbFeat.csv', 'w') as fileWrite:
  myWriter = csv.writer(fileWrite) 
  for i in dbFeat: 
      myWriter.writerow(i)
    
with open('/gdrive/My Drive/dataset/1fqFeat.csv', 'w') as fileWrite:
  myWriter = csv.writer(fileWrite) 
  for i in qFeat: 
      myWriter.writerow(i)

